## Imports

In [1]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

## Load the Data

In [ ]:
df_games = pd.read_csv('../../data/final/steam_games_final.csv')
df_reviews = pd.read_csv('../../data/final/steam_reviews_final.csv')

## Create User Matrix

In [3]:

df_user_matrix = df_reviews.pivot(index='username', columns='product_id', values='rating_sentiment')

# Games that have not been reviewed by a user must be filled with 0
df_user_matrix.fillna(0, inplace=True)

In [4]:
# View the user matrix
df_user_matrix.head()

product_id,10,20,30,40,50,60,70,80,130,220,...,735570,738060,738380,740470,741670,745760,745880,754530,758230,763410
username,,,,,,,,,,,,,,,,,,,,,
I Need Healing,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Æñç,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
! Taz the Husky,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
!!,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
!!!,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
# Calculate similarity matrix (cosine similarity)
user_similarity = cosine_similarity(df_user_matrix)
user_similarity_df = pd.DataFrame(user_similarity, index=df_user_matrix.index, columns=df_user_matrix.index)

In [6]:
def predict_ratings(user_id, user_item_matrix, user_similarity_df, k=10):
    # Identify top-k similar users
    similar_users = user_similarity_df[user_id].nlargest(k+1).iloc[1:]  # Exclude self-similarity
    # Calculate weighted rating predictions
    weighted_ratings = user_item_matrix.loc[similar_users.index].multiply(similar_users.values, axis=0)
    predicted_ratings = weighted_ratings.sum(axis=0) / similar_users.sum()
    return predicted_ratings


import random
from tabulate import tabulate

# Get a random user ID from the user-item matrix index
random_user_id = random.choice(df_user_matrix.index)

# Generate predicted ratings for this random user
predicted_ratings = predict_ratings(random_user_id, df_user_matrix, user_similarity_df)

# Get the top 5 recommended games
top_recommendations = predicted_ratings.nlargest(5)

recommended_game_ids = top_recommendations.index.astype(int)

# Filter df_games to get the records for these game IDs
recommended_games_details = df_games[df_games['id'].isin(recommended_game_ids)]

# Select the columns you want to display
columns_to_display = ['app_name', 'developer', 'price', 'genres', 'url']

# Prepare the data for tabulation
games_to_display = recommended_games_details[columns_to_display]

# Print the table using tabulate
print(f'Top 5 Recommended Games for {random_user_id}:')
print(tabulate(games_to_display, headers='keys', tablefmt='pretty', showindex=False))

Top 5 Recommended Games for Hansitato:
+-----------------+---------------------------------------------------------------------+-------+----------------------------+--------------------------------------------------------+
|    app_name     |                              developer                              | price |           genres           |                          url                           |
+-----------------+---------------------------------------------------------------------+-------+----------------------------+--------------------------------------------------------+
|    BioShock     |                        2K Boston2K Australia                        | 19.99 |     ['Action', 'RPG']      |    http://store.steampowered.com/app/7670/BioShock/    |
|  Borderlands 2  |               Gearbox SoftwareAspyr (Mac &amp; Linux)               | 19.99 |     ['Action', 'RPG']      | http://store.steampowered.com/app/49520/Borderlands_2/ |
|   BioShock® 2   | 2K Marin2K ChinaDigit